In [1]:
from dotenv import load_dotenv
from crewai import Crew
from tasks import MemoryTasks
from agents import LTMAgents
load_dotenv()


CLEARING MEMORY


True

In [2]:
user_context = "I want to create a meal plan for myself for lunch but Im allergic to fish. Im also a vegetarian?"
subsequent_request = 'can you provide a meal plan for me?'

In [3]:

tasks = MemoryTasks()
agents = LTMAgents()

# create agents
context_agent = agents.context_agent()
memory_agent = agents.memory_agent()
retrieval_agent = agents.retrieval_agent()

# create tasks
context_task = tasks.determine_memory_task(context_agent, user_context)
memory_task = tasks.store_memory(memory_agent)
retrieve_and_combine_relevant_memories = tasks.retrieve_and_combine_relevant_memories(retrieval_agent, subsequent_request)

memory_task.context = [context_task]
    
crew = Crew(
    agents=[
        context_agent, 
        memory_agent,
        retrieval_agent
    ],
    tasks=[
        context_task,
        memory_task,
        retrieve_and_combine_relevant_memories
    ]
)

In [4]:
result = crew.kickoff()



> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: Allergy: Fish
Vegetarian: True

> Finished chain.


> Entering new CrewAgentExecutor chain...
I need to store the provided memory about fish allergy and being a vegetarian in the database. I should use the insert_memory tool to store the property type and property value.

Action: insert_memory
Action Input: {"property_type": "Allergy", "property_value": "Fish"} 

None

Final Answer: The memory stored in the database is:
Property Type: Allergy
Property Value: Fish

> Finished chain.


> Entering new CrewAgentExecutor chain...
I need to retrieve relevant memories from the database based on the user's request for a meal plan. Since the user is asking for a meal plan, it's important to consider any allergies they may have, such as fish.

Action: retrieve_memories
Action Input: {"input_text": "meal plan"} 

['Fish']

Final Answer: 
- Allergy: Fish

> Finished chain.


In [5]:
print(result)

- Allergy: Fish


In [6]:
import asyncio
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI()

def get_response(user_query, memories):

    template = """
    You are a helpful assistant. 
    Answer the following questions considering the user's question.
    You will also be provided a list of properties, preferences and restrictions correlating to the user that you should heavily consider
    when answering the user. These attributes are very important and should not be ignored

    User question: {user_question}
    properties, preferences and restrictions: {memories}
    """

    prompt = ChatPromptTemplate.from_template(template)

    llm = ChatOpenAI()
        
    chain = prompt | llm | StrOutputParser()
    
    return chain.invoke({
        "user_question": user_query,
        "memories": memories
    })


In [11]:
print(get_response(subsequent_request, result))

'Certainly! I can definitely provide a meal plan for you. Since you have an allergy to fish, I will make sure to exclude any fish or fish products from the meal plan. Would you like the meal plan to be focused on any specific dietary preferences or goals, such as vegetarian, low-carb, or high-protein? Let me know any additional information or preferences you have so I can create a personalized meal plan for you.'

In [8]:
from memo_store import MemoStore

store = MemoStore(verbosity=0, reset=False, path_to_db_dir="./tmp/teachable_agent_db")
store.list_memos()


LOADING MEMORY FROM DISK
    Location = ./tmp/teachable_agent_db/uid_text_dict.pkl
LIST OF MEMOS
  ID: 1
    INPUT TEXT: Allergy
    OUTPUT TEXT: Fish


In [9]:
from tools import MemoryTools

mems = MemoryTools.retrieve_memories('meal plan')
print(mems)

['Fish']
